# Process the Orders Data
1. Ingest the data into the data lakehouse - bronze_orders
2. Perform data quality checks and transform the data as requied - silver_orders_clean
3. Apply changes to the orders data - silver_orders

1. Ingest the data into the data lakehouse - bronze_orders

In [0]:
-- This operation requires unity catalog

CREATE OR REFRESH STREAMING TABLE bronze_orders
COMMENT 'Raw orders data ingested from the source system operational data'
TBLPROPERTIES ('quality' = 'bronze')
AS
SELECT
  *,
  _metadata.file_path AS input_file_path,
  CURRENT_TIMESTAMP AS ingestion_timestamp
FROM cloud_files(
  '/Volumes/circuitbox/landing/operational_data/orders',
  'json',
  map('cloudFiles.inferColumntypes', 'true')
);

2. Perform data quality checks and transform the data as required - silver_orders_clean

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_order_clean(
  CONSTRAINT valid_customer_id EXPECT (customer_id IS NOT NULL) ON VIOLATION FAIL UPDATE,
  CONSTRAINT valid_order_id EXPECT (order_id IS NOT NULL) ON VIOLATION FAIL UPDATE,
  CONSTRAINT valid_order_status EXPECT (order_status IS IN ('Pending', 'Shipped', 'Cancelled', 'Completed')),
  CONSTRAINT valid_payment_method EXPECT (payment_method IS IN ('Credit Card', 'Bank Transfer', 'Paypal'))
)
COMMENT 'cleaned orders data'
TBLPROPERTIES('quality', 'true')
AS
SELECT
  order_id,
  customer_id,
  CAST("order_timestamp" AS TIMESTAMP) AS order_timestamp
  payment_method,
  items,
  order_status
FROM STREAM(LIVE.bronze_orders)

3. Explode the items array from order object - silver_orders

In [0]:
CREATE STREAMING TABLE silver_orders
AS
SELECT
  order_id,
  customer_id,
  order_timestamp,
  payment_method,
  order_status,
  item.item_id,
  item.name AS item_name,
  item.price AS item_price,
  item.quantity AS item_quanity,
  item.category AS item_category
FROM (
  SELECT
    order_id,
    customer_id,
    order_timestamp,
    payment_method,
    order_status,
    explode(items) AS item
  FROM STREAM(LIVE.silver_orders_clean)
);
